# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [92]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [78]:
vacay="output_data/cities.csv"
v_df = pd.read_csv(vacay)
v_df

,City,Lat,Lng,Humidity,Cloudiness,WindSpeed,Temps
0,atuona,-9.80,-139.03,75,3,21.54,78.94
1,vaini,-21.20,-175.20,74,20,12.75,80.60
2,punta arenas,-53.15,-70.92,81,90,9.17,48.20
3,ribeira grande,38.52,-28.70,68,42,21.23,61.48
4,kruisfontein,-34.00,24.73,83,100,7.00,68.00
...,...,...,...,...,...,...,...
586,masallatah,32.62,14.00,63,33,9.51,66.06
587,cap-aux-meules,47.38,-61.86,40,0,14.99,33.80
588,muzhi,65.37,64.67,86,39,17.60,35.58
589,cayenne,4.93,-52.33,100,90,3.36,77.00


In [79]:
locations = v_df[["Lat", "Lng"]]
H_rating = v_df["Humidity"].astype(float)

In [80]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights= H_rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [81]:
perfect_spot_df = v_df.loc[(v_df["WindSpeed"]<10)&(v_df["Cloudiness"]==0)&(v_df["WindSpeed"]<10)&(v_df["Temps"].between(75,85))]
perfect_spot_df = perfect_spot_df.reset_index(drop =True, inplace =False)
perfect_spot_df

,City,Lat,Lng,Humidity,Cloudiness,WindSpeed,Temps
0,bowen,-20.02,148.23,65,0,2.24,78.80
1,nioaque,-21.14,-55.83,38,0,3.58,76.68
2,formosa,-26.18,-58.18,31,0,8.05,78.80
3,veraval,20.90,70.37,59,0,6.76,78.51
4,gorontalo,0.54,123.06,81,0,1.50,79.41
5,berbera,10.44,45.01,61,0,0.94,77.74
6,kaduqli,11.02,29.72,36,0,7.49,80.26
7,srivardhan,18.03,73.02,62,0,5.93,76.21
8,karratha,-20.74,116.85,29,0,6.58,82.49
9,lalmohan,22.34,90.74,79,0,8.77,75.15


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [96]:



params = {    
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

for index, row in perfect_spot_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    Url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    try:
        hotel_address = requests.get(Url,params=params)
        hotel_address= hotel_address.json()
        perfect_spot_df.loc[index, "Hotel Name"] = hotel_address["results"][0]["name"]
    except IndexError:
        print("No hotel nearby")
#     pprint(hotel_address)


No hotel nearby


In [97]:
perfect_spot_df

,City,Lat,Lng,Humidity,Cloudiness,WindSpeed,Temps,Hotel Name
0,bowen,-20.02,148.23,65,0,2.24,78.80,Horseshoe Bay Resort
1,nioaque,-21.14,-55.83,38,0,3.58,76.68,Nioaque Novo Hotel
2,formosa,-26.18,-58.18,31,0,8.05,78.80,Casagrande Apart Hotel
3,veraval,20.90,70.37,59,0,6.76,78.51,THE SOMNATH GATEWAY
4,gorontalo,0.54,123.06,81,0,1.50,79.41,Maqna Hotel by Prasanthi
5,berbera,10.44,45.01,61,0,0.94,77.74,Damal Hotel
6,kaduqli,11.02,29.72,36,0,7.49,80.26,فندق الله جابو
7,srivardhan,18.03,73.02,62,0,5.93,76.21,Tranquil Beach Resort
8,karratha,-20.74,116.85,29,0,6.58,82.49,Latitude20 The Dunes Apartments
9,lalmohan,22.34,90.74,79,0,8.77,75.15,Arafat Hotel


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
